## 1.Importing libraries

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## 2. Reading the data

In [2]:
project_dir=r"C:\Users\LENOVO\OneDrive\Desktop\Flight_price_predict_project"
data_dir="DataSets"

In [3]:
def get_datasets(name):
    file_name=f"{name}.csv"
    file_path=os.path.join(project_dir, data_dir, file_name)
    return pd.read_csv(file_path)

In [4]:
flights_df=get_datasets("flight_price")
flights_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [5]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


## 3. Preliminanry Analysis

In [6]:
flights_df.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

##### Columns need to be changes or cleaned
* 'Date_of_Journey'    object    type to be changed,
'Dep_Time'           object    type to be changed,
'Arrival_Time'       object    type to be changed,
* 'Duration' object    contains unnessary info

In [7]:
# number of duplicates
flights_df.duplicated().sum()

np.int64(220)

### Checking and finding duplicates entry

In [8]:
(
    flights_df
    .loc[flights_df.duplicated(keep=False)]
    .sort_values(["Airline","Date_of_Journey","Source","Destination","Dep_Time"])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
1535,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


## 4. Detailed Analysis

### Airline

In [9]:
flights_df.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [10]:
(
    flights_df
    .Airline
    .str.replace("Premium economy", "")
    .str.replace("Business", "")
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Vistara ',
       'Jet Airways ', 'Multiple Carriers ', 'Trujet'], dtype=object)

### Date_of_Journey

### Source

In [11]:
flights_df.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [12]:
flights_df.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Dep_Time

In [13]:
(
    flights_df
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [14]:
pd.to_datetime(flights_df.Dep_Time).dt.time

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17000\3186174700.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights_df.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival_time

In [15]:
# are there any un relevent data in column ?
(
    flights_df
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str.get(1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### Duration

In [16]:
flights_df.Duration.head()

0    2h 50m
1    7h 25m
2       19h
3    5h 25m
4    4h 45m
Name: Duration, dtype: object

In [17]:
(
    flights_df
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [18]:
(
    flights_df
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
    #.unique()
)

6474    5m
Name: Duration, dtype: object

In [19]:
(
    flights_df
    .Duration
    .drop(index=6474)
    .str.split(" ",expand=True)
    .set_axis(["hour","minute"], axis=1)
    .assign(
        hour=lambda df_: (
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minute=lambda df_: (
            df_
            .minute
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

### Stops

In [20]:
(
    flights_df
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","",regex=True)
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

### Additional info

In [21]:
flights_df.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [22]:
for col in flights_df.select_dtypes(include="O").columns:
    print(col) 

Airline
Date_of_Journey
Source
Destination
Route
Dep_Time
Arrival_Time
Duration
Total_Stops
Additional_Info


## 5. Cleaning 

In [23]:
def convert_to_minutes(ser):
    return(
        ser
        .str.split(" ",expand=True)
        .set_axis(["hour","minute"], axis=1)
        .assign(
            hour=lambda df_: (
                df_
                .hour
                .str.replace("h","")
                .astype(int)
                .mul(60)
            ),
            minute=lambda df_: (
                df_
                .minute
                .str.replace("m","")
                .fillna("0")
                .astype(int)
            )
        )
        .sum(axis=1)
    )

In [39]:
# we are overwriting the pandas dataframe. The method we use doesn't affect original dataframe

def clean_data(df):
    return (
        df
        .drop(index=[6474])
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include="O").columns 
        })
        .rename(columns=str.lower)
        .assign(
            airline= lambda df_: (
                df_
                .airline
                .str.replace("Premium economy", "")
                .str.replace("Business", "")
                .str.title()
                .str.strip()
            ),
            date_of_journey=lambda df_:pd.to_datetime(df_.date_of_journey, dayfirst=True),
            dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
            arrival_time=lambda df_:pd.to_datetime(df_.arrival_time).dt.time,
            duration=lambda df_:df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_: (
                df_
                .total_stops
                .replace("non-stop","0")
                .str.replace(" stops?","",regex=True)
                .pipe(lambda ser: pd.to_numeric(ser))
                ),
            additional_info=lambda df_: df_.additional_info.replace("No info","No Info")
            )
        .drop(columns="route")
    )
        


In [40]:
clean_data(flights_df).head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17000\2438378219.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17000\2438378219.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_:pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302


In [41]:
flights_cleaned_df=clean_data(flights_df)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17000\2438378219.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17000\2438378219.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_:pd.to_datetime(df_.arrival_time).dt.time,


In [42]:
flights_cleaned_df["airline"].unique()

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

### 6. Split the Data

In [43]:
X=flights_cleaned_df.drop(columns="price")
Y=flights_cleaned_df.price

In [29]:
Y

0         3897
1         7662
2        13882
3         6218
4        13302
         ...  
10678     4107
10679     4145
10680     7229
10681    12648
10682    11753
Name: price, Length: 10462, dtype: int64

In [30]:
X.tail()

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info
10682,Air India,2019-05-09,Delhi,Cochin,10:55:00,19:15:00,500,2.0,No Info


In [44]:
#Train-test split
x_,x_test,y_,y_test=train_test_split(X,Y, test_size=0.2, random_state=42)

x_train,x_val,y_train,y_val=train_test_split(x_,y_, test_size=0.2, random_state=42)


In [45]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)
print(x_.shape, y_.shape)

(6695, 9) (6695,)
(1674, 9) (1674,)
(2093, 9) (2093,)
(8369, 9) (8369,)


### 7. Exporting the Subsets

In [46]:
def export_data(x,y,name):
    file_name=f"{name}.csv"
    file_path=os.path.join(project_dir, data_dir, file_name)

    x.join(y).to_csv(file_path, index=False)
    return pd.read_csv(file_path).head()

In [47]:
export_data(x_train,y_train,"train1_dataset")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-21,Banglore,New Delhi,08:55:00,19:10:00,615,1.0,In-flight meal not included,7832
1,Jet Airways,2019-03-27,Delhi,Cochin,17:30:00,04:25:00,655,1.0,In-flight meal not included,6540
2,Goair,2019-03-09,Banglore,New Delhi,11:40:00,14:35:00,175,0.0,No Info,7305
3,Air India,2019-06-12,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No Info,8366
4,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,07:40:00,525,1.0,In-flight meal not included,11087


In [48]:
export_data(x_test,y_test,"test1_dataset")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,08:15:00,1455,1.0,No Info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0.0,No Info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0.0,No Info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0.0,No Info,3597
4,Indigo,2019-05-06,Kolkata,Banglore,15:15:00,17:45:00,150,0.0,No Info,4804


In [49]:
export_data(x_val,y_val,"Val1_dataset")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1.0,No Info,5054
1,Multiple Carriers,2019-06-12,Delhi,Cochin,09:45:00,22:30:00,765,1.0,No Info,9646
2,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,15:15:00,980,1.0,In-flight meal not included,11087
3,Multiple Carriers,2019-06-06,Delhi,Cochin,13:00:00,21:00:00,480,1.0,No Info,13587
4,Jet Airways,2019-05-18,Delhi,Cochin,23:05:00,04:25:00,1760,2.0,No Info,16704
